In [1]:
using LinearAlgebra

In [2]:
function barrier(P, ∇P, x, c; μ=1)
    tol = 1e-6  # stopping tolerance for μ
    k = 1
    while μ > tol && k <= 100
        x = steepest_descent_barrier(P, ∇P, x, μ, c) #minimize a log barrier funct. using steepest descent
        μ *= 0.5 #reduce barrier param
        k += 1
    end

    return x, μ
end

barrier (generic function with 1 method)

In [3]:
function backtracking_line_search_barrier(f, ∇f, x, μ, c, p)
    α = .5     # initial step length
    rho = 0.75
    ctol = 1e-4

    function stay_feasible(α)
        while c(x + α*p) < 0
            α *= rho
        end
        return α
    end
    α = stay_feasible(α)

    while f(x + α*p, μ) > f(x, μ) + ctol*α*transpose(∇f(x, μ))*p 
        α *= rho
        α = stay_feasible(α)
    end
    return α
end

backtracking_line_search_barrier (generic function with 1 method)

In [4]:
function steepest_descent_barrier(f, ∇f, x, μ, c; ϵ=1e-3, k=1, c1=1e-4)
    i = 1
    while norm(∇f(x, μ)) > ϵ && i <= 1000
        p = -∇f(x, μ)  #steep descent direction calculation
        α = backtracking_line_search_barrier(f, ∇f, x, μ, c, p) #use backtracking to update step size and feasibility
        x = x + α * p #iterate update
        if i % 50 == 0 #print progress
            println("Iteration ", i, ": x = ", x)
        end

        i += 1 #update iteration #
    end

    return x
end

steepest_descent_barrier (generic function with 1 method)

In [5]:
f(x) = (x[1]+0.5)^2 + (x[2]-0.5)^2
c(x) = x[1]
B(x) = -log(c(x))
P(x,μ) = f(x) + μ*B(x)
∇P(x,μ) = [ 2*(x[1]+0.5)-μ/x[1], 2*(x[2]-0.5) ]  # need this for steepest descent

x0 = [1,1]   # start at a feasible point
(x,μ) = barrier(P, ∇P, x0, c; μ=1.0)
println("solution occurs near x = ", x)

Iteration 100: x = [0.18325791063799937, 0.5000000000000001]
Iteration 200: x = [0.18318246209431743, 0.5000000000000001]
Iteration 300: x = [0.18313024732927505, 0.5000000000000001]
Iteration 100: x = [1.910735233669821e-6, 0.5000000000000001]
solution occurs near x = [1.9054389041086638e-6, 0.5000000000000001]
